In [1]:
!pip install spacy-wordnet


[notice] A new release of pip available: 23.0 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np
from collections import Counter

In [2]:
os.getcwd()

'C:\\Users\\soumi\\Desktop\\SRMIST_OD203SRM_Document_classification\\ML_DL Notebooks'

In [3]:

os.getcwd()
entertainment=  pd.read_csv('../Dataset/Entertainment/Entertainment_Dataset.csv')
entertainment.columns = ['unnamed', 'values', 'category']
entertainment = entertainment.drop('unnamed', axis = 1)

insurance=  pd.read_csv('../Dataset/insurance/insurance_dataset.csv')
insurance.columns = ['unnamed', 'values', 'category']
insurance = insurance.drop('unnamed', axis = 1)

finance=  pd.read_csv('../Dataset/finance/finance_dataset.csv')
finance.columns = ['unnamed', 'values', 'category']
finance = finance.drop('unnamed', axis = 1)

travel = pd.read_csv('../Dataset/travel/Travel_Dataset.csv')
travel.columns = ['unnamed', 'values', 'category']
travel = travel.drop('unnamed', axis = 1)

medical = pd.read_csv('../Dataset/Medical/medical_dataset.csv')
medical.columns = ['unnamed', 'values', 'category']
medical = medical.drop('unnamed', axis = 1)

l = [insurance, entertainment, finance, travel, medical]
df = pd.concat(l)

In [4]:
df.head()
Counter(df['category'])

Counter({'insurance': 1000,
         'entertainment': 386,
         'finance': 44999,
         nan: 10,
         'travel': 957,
         'medical': 13200})

In [6]:

from textgenie import TextGenie
from tqdm import tqdm
textgenie = TextGenie(
                    "hetpandya/t5-small-tapaco",
                    "bert-base-uncased",
                    "en_core_web_sm",
                    device = 'cuda:0'
                     )

def augmentor(dataset):
    es = list(eval(f"{dataset}['values']"))
    aug = []
    for x in tqdm(range(len(es))):
        es[x] = es[x].replace('\n', ' ')
        es[x] = es[x].replace("\'", ' ')
        try:
            l = textgenie.magic_once(
                                    f"{es[x]}",
                                    "paraphrase: ",
                                    n_paraphrase_predictions=2 ,
                                    n_mask_predictions=1,
                                    convert_to_active = False
                                    )
            
            aug.extend(l)
        except:
            continue
    es.extend(aug)
    return es

[nltk_data] Error loading wordnet: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading wordnet: HTTP Error 404: Not Found


ValueError: Error loading wordnet: HTTP Error 404: Not Found

# It  takes about 12 hours to run below code on GPU !!

In [7]:
# ent = augmentor('entertainment')
# tra = augmentor('travel')
# ins = augmentor('insurance')

In [8]:
# def to_dataset(listname, category):
#     a = eval(listname)
#     df = pd.DataFrame(zip(a, [category]*len(a)) ,columns = ['values', 'category'])
#     return df
# to_dataset('ent', 'entertainment').to_csv('Dataset/Entertainment/entertainment_aug.csv')
# to_dataset('tra', 'travel').to_csv('Dataset/Travel/travel_aug.csv')
# to_dataset('ins', 'insurance').to_csv('Dataset/Insurance/insurance_aug.csv')

In [9]:
entertainment = pd.read_csv('../Dataset/Entertainment/entertainment_aug.csv')
entertainment.columns = ['unnamed', 'values', 'category']
entertainment = entertainment.drop('unnamed', axis = 1)

travel = pd.read_csv('../Dataset/Travel/travel_aug.csv')
travel.columns = ['unnamed', 'values', 'category']
travel = travel.drop('unnamed', axis = 1)

insurance = pd.read_csv('../Dataset/Insurance/insurance_aug.csv')
insurance.columns = ['unnamed', 'values', 'category']
insurance = insurance.drop('unnamed', axis = 1)

In [10]:
entertainment

,values,category
0,Gallery unveils interactive tree A Christmas ...,entertainment
1,Jarre joins fairytale celebration French musi...,entertainment
2,Musical treatment for Capra film The classic ...,entertainment
3,Richard and Judy choose top books The 10 auth...,entertainment
4,Poppins musical gets flying start The stage a...,entertainment
...,...,...
20872,buffy creator joins wonder woman the creator o...,entertainment
20873,buffy creator joins wonder woman the creator o...,entertainment
20874,buffy creator joins wonder woman the creator o...,entertainment
20875,buffy creator joins wonder woman the creator o...,entertainment


In [11]:
l = [insurance, entertainment, finance, travel, medical]
df = pd.concat(l)

In [12]:
df

,values,category
0,can you borrow against globe Life Insurancebor...,insurance
1,do Medicare cover my spouseif your spouse have...,insurance
2,what happen when you change homeowner insuranc...,insurance
3,what be a typical renter insurance costI be su...,insurance
4,what be car insurance base oncar insurance rat...,insurance
...,...,...
13195,"([""Intact function of the Forkhead Box P2 (FOX...",medical
13196,"([""Studies on ADHD in educational settings ind...",medical
13197,(['The mechanisms underlying cerebellar learni...,medical
13198,(['Withania somnifera root extract has been us...,medical


In [13]:
print(Counter(df['category']))
categories = list(dict(Counter(df['category'])).keys())

Counter({'finance': 44999, 'travel': 38451, 'insurance': 25801, 'entertainment': 20877, 'medical': 13200, nan: 10})


In [14]:
categories

['insurance', 'entertainment', 'finance', nan, 'travel', 'medical']

In [15]:
df['values']

0        can you borrow against globe Life Insurancebor...
1        do Medicare cover my spouseif your spouse have...
2        what happen when you change homeowner insuranc...
3        what be a typical renter insurance costI be su...
4        what be car insurance base oncar insurance rat...
                               ...                        
13195    (["Intact function of the Forkhead Box P2 (FOX...
13196    (["Studies on ADHD in educational settings ind...
13197    (['The mechanisms underlying cerebellar learni...
13198    (['Withania somnifera root extract has been us...
13199    (['Deep brain stimulation (DBS) has been found...
Name: values, Length: 143338, dtype: object

In [16]:
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['values'])
tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(X)
print(X.shape)

(143338, 196738)


In [17]:
mp = {categories[x]:x for x in range(len(categories))}

y_true = []
for x in df['category']:
    y_true.append(mp[x])

In [18]:
import numpy as np
y_true = np.array(y_true)
y_true.shape

(143338,)

In [26]:
def TrainSplitEqualBinary(X, y, samples_n): #samples_n per class
    
    indicesClass1 = []
    indicesClass2 = []
    
    for i in range(0, len(y)):
        if y[i] == 0 and len(indicesClass1) < samples_n:
            indicesClass1.append(i)
        elif y[i] == 1 and len(indicesClass2) < samples_n:
            indicesClass2.append(i)
            
        if len(indicesClass1) == samples_n and len(indicesClass2) == samples_n:
            break
    
    X_test_class1 = X[indicesClass1]
    X_test_class2 = X[indicesClass2]
    
    X_test = np.concatenate((X_test_class1,X_test_class2), axis=0)
    
    #remove x_test from X
    X_train = np.delete(X, indicesClass1 + indicesClass2, axis=0)
    
    Y_test_class1 = y[indicesClass1]
    Y_test_class2 = y[indicesClass2]
    
    y_test = np.concatenate((Y_test_class1,Y_test_class2), axis=0)
    
    #remove y_test from y
    y_train = np.delete(y, indicesClass1 + indicesClass2, axis=0)
    
    if (X_test.shape[0] != 2 * samples_n or y_test.shape[0] != 2 * samples_n):
        raise Exception("Problem with split 1!")
        
    if (X_train.shape[0] + X_test.shape[0] != X.shape[0] or y_train.shape[0] + y_test.shape[0] != y.shape[0]):
        raise Exception("Problem with split 2!")
    
    return X_train, X_test, y_train, y_test

In [28]:
X_train, X_test, y_train, y_test = TrainSplitEqualBinary(X, y_true, samples_n=3300)

ValueError: zero-dimensional arrays cannot be concatenated

In [ ]:
# Train the model
clf = LogisticRegression(max_iter = 300)
clf.fit(X_train, y_train)

In [20]:
# Evaluate the model
from sklearn.metrics import balanced_accuracy_score as bas
from sklearn.metrics import roc_auc_score as ras
from sklearn.metrics import confusion_matrix as cm

y_pred = clf.predict(X_test)

accuracy = bas(y_test, y_pred)
print(f"Accuracy: {accuracy}")

conf_m = cm(y_test, y_pred)


y_pred = clf.predict_proba(X_test)
accuracy = ras(y_test, y_pred, multi_class='ovr')
print(f"roc_auc_score: {accuracy}")

print(f"confusion matrix: \n {conf_m}")

Accuracy: 0.8320242993294364
roc_auc_score: 0.9283707493431214
confusion matrix: 
 [[5140    0    6    0    1    1]
 [   0 4285    3    0    0    0]
 [   4    0 8847    0    0    0]
 [   0    0    2    0    0    0]
 [   3    1   15    0 7779    0]
 [   0    0    7    0    0 2574]]


In [21]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(X_train, y_train)

BernoulliNB()

In [22]:
y_pred = clf.predict(X_test)

accuracy = bas(y_test, y_pred)
print(f"Accuracy: {accuracy}")

conf_m = cm(y_test, y_pred)


y_pred = clf.predict_proba(X_test)
accuracy = ras(y_test, y_pred, multi_class='ovr')
print(f"roc_auc_score: {accuracy}")

print(f"confusion matrix: \n {conf_m}")

Accuracy: 0.7933499810118777
roc_auc_score: 0.9154886641358875
confusion matrix: 
 [[5143    0    0    0    5    0]
 [  23 4233    1    0   31    0]
 [  58    0 7275    0 1511    7]
 [   0    0    1    0    1    0]
 [   0    0    0    0 7798    0]
 [  75    1   32    0   16 2457]]


In [23]:
from sklearn import svm
from sklearn.calibration import CalibratedClassifierCV
clf = CalibratedClassifierCV(svm.LinearSVC())
clf.fit(X_train, y_train)

CalibratedClassifierCV(base_estimator=LinearSVC())

In [24]:
y_pred = clf.predict(X_test)

accuracy = bas(y_test, y_pred)
print(f"Accuracy: {accuracy}")

conf_m = cm(y_test, y_pred)


y_pred = clf.predict_proba(X_test)
accuracy = ras(y_test, y_pred, multi_class='ovr')
print(f"roc_auc_score: {accuracy}")

print(f"confusion matrix: \n {conf_m}")

Accuracy: 0.9158080072141971
roc_auc_score: 0.9998033333674085
confusion matrix: 
 [[5145    0    1    0    1    1]
 [   0 4288    0    0    0    0]
 [   3    0 8847    0    0    1]
 [   0    0    1    1    0    0]
 [   4    1   12    0 7781    0]
 [   0    0    5    0    0 2576]]


In [25]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
clf = CalibratedClassifierCV(make_pipeline(StandardScaler(with_mean=False),
                    SGDClassifier(max_iter=1000, tol=1e-3)))
clf.fit(X_train, y_train)

CalibratedClassifierCV(base_estimator=Pipeline(steps=[('standardscaler',
                                                       StandardScaler(with_mean=False)),
                                                      ('sgdclassifier',
                                                       SGDClassifier())]))

In [26]:
y_pred = clf.predict(X_test)

accuracy = bas(y_test, y_pred)
print(f"Accuracy: {accuracy}")

conf_m = cm(y_test, y_pred)

y_pred = clf.predict_proba(X_test)
accuracy = ras(y_test, y_pred, multi_class='ovr')
print(f"roc_auc_score: {accuracy}")

print(f"confusion matrix: \n {conf_m}")

Accuracy: 0.8038519033641837
roc_auc_score: 0.9564457044123554
confusion matrix: 
 [[5093    1   50    0    3    1]
 [   3 4134  147    0    4    0]
 [   0    5 8827    0   19    0]
 [   0    0    2    0    0    0]
 [   8    1   13    0 7776    0]
 [   1    3  317    0    1 2259]]


# Best accuracy is given by the LinearSVM classifier.

In [27]:
from sklearn import svm
from sklearn.calibration import CalibratedClassifierCV
clf = CalibratedClassifierCV(svm.LinearSVC())
clf.fit(X_train, y_train)

CalibratedClassifierCV(base_estimator=LinearSVC())

In [28]:
new_data = [
            'flights from mumbai to japan',
           '5000 rupees dollar to be transferred tommorow',
           'health insurance is realy expensive',
           'the Lord Of The Rings is a great movie',
           'liver surgery is needed',
           'The largest organ on the body is the skin also known as derma',
            'the train is late again',
            'let us get on the bus',
            'we can go to the train station using the car',
           ]
X_new = vectorizer.transform(new_data)
X_new = tfidf_transformer.transform(X_new)
y_new = clf.predict(X_new)
for x in y_new:
    print(categories[x])

travel
finance
insurance
entertainment
medical
finance
travel
travel
travel


In [29]:
finance_news = ['Also known as a credit document.\
                This term usually refers to the main documents in a financing\
                transaction under which an obligor owes financial obligations to\
                (or which otherwise create a liability in favour of) a lender (or lenders),\
                agent, arranger or other secured party (for example, a swap counterparty).\
                It is often used as a defined term and, although the parties may designate\
                any document a finance document, it typically includes: Facility agreements.\
                Security documents (such as mortgages and charges). Guarantees and indemnities.\
                Agreements appointing security trustees. Swaps and other derivative contracts.\
                Priority and ranking agreements (such as intercreditor agreements). Drawdown requests.\
                Fee letters.']

print('Number of words in above sentence = ', len(finance_news[0].split()))
X_new = vectorizer.transform(finance_news)
X_new = tfidf_transformer.transform(X_new)
y_new = clf.predict(X_new)
for x in y_new:
    print(categories[x])

Number of words in above sentence =  105
finance


In [30]:
travel_news = ["Firstly, you might want to consider visiting some\
                of India's most popular destinations such as Delhi, Agra, and Jaipur,\
                collectively known as the Golden Triangle. These cities are filled \
                with stunning monuments, palaces, and forts that will give you a glimpse\
                into India's rich history and culture. In Delhi, you could visit the Red Fort,\
                India Gate, Lotus Temple, and the Qutub Minar, to name just a few of the many \
                amazing sights in the city. Agra is home to the iconic Taj Mahal, one of the\
                Seven Wonders of the World, and also the Agra Fort. In Jaipur, you could explore\
                the beautiful City Palace, Hawa Mahal, and the Amber Fort."]

print('Number of words in above sentence = ', len(travel_news[0].split()))
X_new = vectorizer.transform(travel_news)
X_new = tfidf_transformer.transform(X_new)
y_new = clf.predict(X_new)
for x in y_new:
    print(categories[x])

Number of words in above sentence =  112
travel


In [31]:
medical = ["The heart is a muscular organ located \
            in the chest cavity that is responsible for pumping\
            blood throughout the body. The heart has four chambers \
            - two atria and two ventricles - that work together to \
            circulate blood. The process of how the heart functions \
            can be described in the following steps: Deoxygenated blood\
            from the body enters the right atrium through the superior\
            and inferior vena cava. The right atrium contracts, pushing \
            blood through the tricuspid valve into the right ventricle. \
            The right ventricle contracts, pumping blood through the pulmonary\
            valve into the pulmonary artery, which carries the blood to the lungs.\
            In the lungs, the blood receives oxygen and releases carbon dioxide.\
            Oxygenated blood from the lungs returns to the heart via the pulmonary\
            veins, entering the left atrium. The left atrium contracts, pushing blood \
            through the mitral valve into the left ventricle. The left \
            ventricle contracts, pumping blood through the aortic valve into\
            the aorta, which carries the blood to the rest of the body. The \
            cycle repeats as deoxygenated blood returns to the right atrium. \
            The heart's pumping action is controlled by electrical impulses\
            that originate in the sinoatrial (SA) node, a specialized group \
            of cells in the right atrium. These electrical impulses spread \
            through the heart, causing the chambers to contract in a coordinated \
            manner, which results in an efficient circulation of \
            blood throughout the body."]

print('Number of words in above sentence = ', len(medical[0].split()))
X_new = vectorizer.transform(medical)
X_new = tfidf_transformer.transform(X_new)
y_new = clf.predict(X_new)
for x in y_new:
    print(categories[x])

Number of words in above sentence =  231
medical


In [32]:
insurance = ["Insurance is a contract \
            between an individual or entity (policyholder) \
            and an insurance company (insurer) in which the\
            policyholder pays a premium in exchange for protection\
            against specific risks or losses. Here are some\
            important facts about insurance: Types of Insurance: \
            There are many types of insurance, including health\
            insurance, life insurance, car insurance, home insurance,\
            and business insurance, among others. Premiums: \
            Insurance policies require the policyholder to pay\
            a premium, which is a periodic payment (monthly,\
            quarterly, annually, etc.) to maintain the policy.\
            Premiums are typically based on the level of risk\
            associated with the insured individual or entity.\
            Deductibles: A deductible is the amount the\
            policyholder is responsible for paying before \
            insurance coverage kicks in. For example, in car\
            insurance, if you have a $500 deductible and you \
            get into an accident that causes $1,500 worth \
            of damage, you would pay the first $500 and \
            the insurance company would cover the remaining\
            $1,000. Coverage Limits: Insurance policies also\
            have coverage limits, which are the maximum amounts\
            that the insurance company will pay out in the\
            event of a covered loss or claim. Policyholders \
            can choose coverage limits based on their \
            individual needs and budget. Claims: In the\
            event of a loss or claim, the policyholder must\
            file a claim with the insurance company. The insurer\
            will investigate the claim and determine if it is \
            covered under the policy. If approved, the insurer will\
            provide compensation or reimbursement for the loss, up \
            to the coverage limit. Benefits: Insurance provides financial\
            protection and peace of mind for policyholders in the \
            event of unexpected losses or damages. It can also help\
            to mitigate risks associated with certain activities or\
            situations, such as driving a car or owning a home.\
            Legal Requirements: Some types of insurance are legally \
            required, such as car insurance in most states, while others \
            are optional but highly recommended, such as health insurance.\
            It's important to understand your insurance policy and the level\
            of coverage it provides to ensure you have the appropriate protection\
            in place for your specific needs."]

print('Number of words in above sentence = ', len(insurance[0].split()))
X_new = vectorizer.transform(insurance)
X_new = tfidf_transformer.transform(X_new)
y_new = clf.predict(X_new)
for x in y_new:
    print(categories[x])

Number of words in above sentence =  342
insurance


In [33]:
entertainment = ["Titanic is a classic movie that was released in \
                1997 and directed by James Cameron. The film is a fictionalized \
                account of the tragic sinking of the Titanic, a luxury liner that \
                hit an iceberg and sank on its maiden voyage in 1912. The movie is\
                a visually stunning masterpiece that tells a compelling love story\
                between two passengers from different social classes who fall in\
                love aboard the ship. The performances by the lead actors, Leonardo\
                DiCaprio and Kate Winslet, are exceptional and their chemistry on-screen\
                is truly captivating. The film's cinematography and special effects are\
                also impressive, especially in the scenes depicting the sinking of the\
                ship, which are both dramatic and heartbreaking. One of the strengths\
                of the film is its attention to historical detail and accuracy, from \
                the design of the ship to the costumes worn by the characters. The film \
                also sheds light on the social dynamics of the time, highlighting the class\
                divisions that existed and the struggles faced by immigrants seeking a better\
                life in America. Overall, Titanic is a classic and unforgettable film that has \
                stood the test of time. Its masterful storytelling, stunning visuals, \
                and powerful performances make it a must-see for any movie lover, and its \
                themes of love, loss, and sacrifice continue to resonate with audiences today."]
print('Number of words in above sentence = ', len(entertainment[0].split()))
X_new = vectorizer.transform(entertainment)
X_new = tfidf_transformer.transform(X_new)
y_new = clf.predict(X_new)
for x in y_new:
    print(categories[x])

Number of words in above sentence =  218
entertainment


In [55]:
(vectorizer)
tfidf_transformer
import pickle

In [56]:
file = open('vectorizer', 'wb')
pickle.dump(vectorizer, file)

file = open('tfidf_transformer', 'wb')
pickle.dump(tfidf_transformer, file)

In [57]:
with open('vectorizer', 'wb') as fp:
    pickle.dump(vectorizer, fp)

In [58]:
with open('tfidf_transformer', 'wb') as fp:
    pickle.dump(tfidf_transformer, fp)